In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random

In [9]:
model_id = "deepseek-ai/deepseek-coder-1.3b-instruct"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading tokenizer...


In [10]:
print("Loading model (CPU, float16)...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="cpu",   # CPU-only inference
)

Loading model (CPU, float16)...


In [11]:
from prompt_generator import CodeReviewPromptGenerator
prompt_generator = CodeReviewPromptGenerator()

In [ ]:
# calcualte avg max token
import os
import json

datasets = os.listdir('./synthetic_datasets')

max_tokens_code_review_result = []
max_tokens_code_review_prompt = []
prompts = []
for dataset in datasets:
    with open(f'./synthetic_datasets/{dataset}', 'r') as f:
        data = json.load(f)
        for i in data:
            max_tokens_code_review_result.append(len(i['code_review_suggestion']))
            prompt = prompt_generator.generate_style_review_prompt(
                added_code=i['added_code'],
                deleted_code=i['deleted_code'],
                full_function_code=i['full_function_code'],
                function_name=i['function_name'],
            )
            max_tokens_code_review_prompt.append(len(prompt))
            prompts.append(prompt)

avg_max_tokens_code_review_result = sum(max_tokens_code_review_result) / len(max_tokens_code_review_result) if max_tokens_code_review_result else 0
avg_max_tokens_code_review_prompt = sum(max_tokens_code_review_prompt) / len(max_tokens_code_review_prompt) if max_tokens_code_review_prompt else 0

In [13]:
print("avg_max_tokens_code_review_result:", avg_max_tokens_code_review_result)
print("avg_max_tokens_code_review_prompt:", avg_max_tokens_code_review_prompt)

avg_max_tokens_code_review_result: 466.73426573426576
avg_max_tokens_code_review_prompt: 1271.8592657342658


In [ ]:
prompts = random.choices(prompts, k=5)



In [22]:
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.9,         # slightly higher → more creative
            top_p=0.9,               # nucleus sampling
            do_sample=True,          # allows stochastic sampling
            repetition_penalty=1.2,  # discourages exact phrase repetition
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        print("\n--- Prompt ---\n")
        print(prompt)
        print("\n--- Generated Output ---\n")
        print(tokenizer.decode(outputs[0], skip_special_tokens=True))
        break


--- Prompt ---

You are a code reviewer. Analyze this Python code change and respond EXACTLY in the format below.

Full function `optimize_similar_text_search`:
```python
def optimize_similar_text_search(query, documents):
    cache = {}
    def cache_results(func):
        def wrapper(*args):
            if args in cache:
                return cache[args]
            result = func(*args)
            cache[args] = result
            return result
        return wrapper
    @cache_results
    def search_document(document):
        return document.count(query)
    results = [search_document(document) for document in documents]
    return results
```
ADDED:
```python
def cache_results(func):
        def wrapper(*args):
            if args in cache:
                return cache[args]
            result = func(*args)
            cache[args] = result
            return result
```
REMOVED:
```python
results = [document.count(query) for document in documents]
```


You MUST respond in this E

In [25]:
prompt = "you are a senior software engineer, create function to add two numbers in python"
inputs = tokenizer(prompt, return_tensors="pt")
with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.9,         # slightly higher → more creative
        top_p=0.9,               # nucleus sampling
        do_sample=True,          # allows stochastic sampling
        repetition_penalty=1.2,  # discourages exact phrase repetition
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )


In [27]:
output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n--- Prompt ---\n")
print(prompt)
print("\n--- Generated Output ---\n")
print(output)


--- Prompt ---

you are a senior software engineer, create function to add two numbers in python

--- Generated Output ---

you are a senior software engineer, create function to add two numbers in python
    def add(x , y):  #defining the functions
        return x +y   // returning addition of both number inputs.     ```python# Adding Code Here ```print("Addition: ",add())This will not work since 'request' is undefined when run on an interactive Python environment like Jupython notebook or IDLE (it does exist inside scripts but isn’t available as partof any module)If you have code that needs input from this user at runtime then it might look something along these lines - "Enter your name:"name = raw_input() print('Hello %s!'%nme Hello world program by Alex Kudelski-Touheeb and Alex Blewitt also known simply referred just hello World Coding practice for beginners here we try our first coding experience with printing out messages such programming concepts can be challenging sometimes 